In [37]:
import numpy as np
import matplotlib.pyplot as pyplot
from scipy.integrate import dblquad
from sympy import *
x = symbols('x')
z = symbols('z')


#PARAMETERS

E = 75e9           # [Pa] Young's Modulus
v = 0.33           # Poissons Ratio
G = E/(2*(1+v))
First  = E*(1-v)/((1+v)*(1-2*v))
Second = v*E/((1+v)*(1-2*v))
C_11 = First
C_22 = First
C_33 = First
C_12 = Second
C_13 = Second
C_23 = Second
C_44 = G
C_55 = G
C_66 = G
                           
#DO not forget the jacobian for integrating you dip shit
    
K =  C_11*6.25*0.01*integrate((2*x*z**2 - 2*x*z - z**2 + z)*(2*x*z**2 - 2*x*z - z**2 + z) + z,(x,-1, 1),(z,-1,1))*(2/3) + C_66*0.01*6.25*integrate((2*x*z**2 - 2*x*z - z**2 + z)*(2*x*z**2 - 2*x*z - z**2 + z),(x,-1, 1),(z,-1,1))*(2/3) + C_55*(1/16)*0.01*integrate((x**2 - x) * (z**2 - z )* (x**2 - x) * (z**2 - z) ,(x,-1, 1),(z,-1,1))*(1/2)
print(K) 


28905793896.5060
